In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data and display first 5 rows
z_data = pd.read_csv('data_task_groups_roi_deact.csv',sep=';',decimal='.')

#== begin create labels =========================================================================
#we need the list with the labels; everytime the label or proband changes, we add it to the list
labels = []

condition = z_data.at[0, 'task']
participant = z_data.at[0, 'proband']
    
for i in range(len(z_data)):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        if condition == 'C':
            labels.append(0)
        if condition == 'R':
            labels.append(1)
        if condition == 'S':
            labels.append(2)
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
labels.append(1)
#== end create labels =========================================================================

#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== now create the different groups and compute the groupwise mean ============================
grouped = z_data.groupby(['proband', 'trial','task'])
groupedAgg = grouped.aggregate(np.mean)
#=== end aggregation ===========================================================================

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = list(groupedAgg.columns)

# Convert to numpy array
features = np.array(groupedAgg)
labels = np.array(labels)

#get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
training_features = features[0:588]
training_features = np.array(training_features)
testing_features = features[588:784]
testing_features = np.array(testing_features)

training_target = labels[0:588]
training_target = np.array(training_target)
testing_target = labels[588:784]
testing_target = np.array(testing_target)
#=== end creating training and validation set ===================================================


In [4]:
from tpot import TPOTClassifier
fileNameTPot = 'allROIS/tpot_mnist_pipeline_FineAverageParticipantSplit_Act'

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs = 20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

Generation 1 - Current best internal CV score: 0.49662391924055205



Best pipeline: GradientBoostingClassifier(RBFSampler(input_matrix, gamma=0.8500000000000001), learning_rate=0.01, max_depth=5, max_features=0.7500000000000001, min_samples_leaf=16, min_samples_split=18, n_estimators=100, subsample=0.3)
0.5


True

In [5]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [6]:
%run -i $fileForLearning

In [9]:
num_matches = 0;
for a, b in zip(testing_target, results):
    if a == b:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',testing_target.size,')')

accuracy = num_matches/testing_target.size*100
print('Accuary: ',accuracy)


Number of matches: 91 (of 180 )
Accuary:  50.55555555555556


In [10]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'Fine_AllROIs' + str(accuracy) + '\n')
